### Day 16
#### part A

In [18]:
import numpy as np
import pandas as pd
import re

In [3]:
infile="16a_input.txt"

In [10]:
with open(infile) as f:
    lines=f.readlines()

In [44]:
phase=0 # 0: conditions, 1: mynumber, 2:othernumbers

conditions=dict()
mynumbers=[]
othernumbers=[]

for idx,line in enumerate(lines):
    if phase==0: # conditions
        if line=="\n":
            phase=1
            continue
        textfield=line.split(":")[0]
        nrs=[int(i) for i in re.findall(r"\d+",line)] # 4 nrs
        cond1=nrs[:2]
        cond2=nrs[2:]
        conditions[textfield]=[cond1,cond2]
        
    if phase==1: # my number
        if re.search(r"\d+",line) is not None: # has numbers
            line=line.replace("\n","")
            mynumbers=[int(l) for l in line.split(",")]
        if line=="\n":
            phase=2
            continue

    if phase==2: # other numbers
        if re.search(r"\d+",line) is not None: # has numbers
            line=line.replace("\n","")
            nrs=[int(l) for l in line.split(",")]
            othernumbers.append(nrs)

In [52]:
def test_conditions(conditions,nrs):
    # returns the sum of the invalid numbers in one ticket
    sum_invalid=0
    for nr in nrs:
        nr_is_valid=False
        for key in conditions.keys():
            conds=conditions[key]
            for cond in conds: # conds: [[100,200],[250,400]] etc
                if nr>=cond[0] and nr<=cond[1]:
                    nr_is_valid=True
        if nr_is_valid==False:
            sum_invalid+=nr
    return sum_invalid

In [55]:
# test all other numbers
sum_invalid=0
for othernumber in othernumbers:
    sum_invalid+=test_conditions(conditions,othernumber)

In [56]:
sum_invalid

23054

#### part B

In [58]:
# remove the invalid tickets
cleanothers=[]
for othernumber in othernumbers:
    if test_conditions(conditions,othernumber)==0:
        cleanothers.append(othernumber)

In [63]:
df=pd.DataFrame(cleanothers)
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,153,109,923,689,426,793,483,628,843,774,785,841,63,168,314,725,489,339,231,914
1,98,718,599,348,225,261,310,490,773,867,659,874,286,290,408,481,780,240,309,391
2,523,86,241,489,735,854,797,654,380,846,522,432,87,911,849,88,219,757,564,901
3,202,581,555,337,185,699,857,222,869,826,873,445,273,440,465,938,766,482,901,99
4,793,468,773,432,300,82,853,557,408,75,220,468,610,748,51,415,775,154,784,189


In [64]:
def test_one_condition_list(conds,nr):
    # conds: [[100,200],[250,400]] etc
    nr_is_valid=False
    for cond in conds: 
        if nr>=cond[0] and nr<=cond[1]:
            nr_is_valid=True
    return nr_is_valid


In [71]:
conditions.keys()

dict_keys(['departure location', 'departure station', 'departure platform', 'departure track', 'departure date', 'departure time', 'arrival location', 'arrival station', 'arrival platform', 'arrival track', 'class', 'duration', 'price', 'route', 'row', 'seat', 'train', 'type', 'wagon', 'zone'])

In [78]:
totalcount=df.shape[0]
totalcount

190

how to find the columns where all tickets fullfill the condition - an example

In [81]:
out=df.applymap(lambda x: test_one_condition_list(conditions["departure location"],x))
out.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True
1,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True
2,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True
3,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True
4,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True


In [88]:
# column idx with all True
np.where(out.all())[0]

array([ 2,  5,  6,  7,  8,  9, 10, 11, 14, 18, 19], dtype=int64)

full run

In [97]:
keys=list(conditions.keys())
candidates=[]
lengths=np.zeros(len(keys))
for idx,key in enumerate(conditions.keys()):
    conds=conditions[key]
    out=df.applymap(lambda x: test_one_condition_list(conditions[key],x))
    valids=np.where(out.all())[0]
    candidates.append(valids)
    lengths[idx]=len(valids)
    print(key,valids)

departure location [ 2  5  6  7  8  9 10 11 14 18 19]
departure station [ 2  3  4  5  6  7  8  9 10 11 12 14 15 17 18 19]
departure platform [ 2  3  5  6  7  8  9 10 11 14 15 18 19]
departure track [ 2  3  5  6  7  8  9 10 11 14 18 19]
departure date [ 2  3  5  6  7  8  9 10 11 14 15 17 18 19]
departure time [ 2  3  4  5  6  7  8  9 10 11 14 15 17 18 19]
arrival location [ 2  5  6  7  9 10 11 18 19]
arrival station [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]
arrival platform [ 2  5 10]
arrival track [ 2  5  9 10 18]
class [ 0  2  3  4  5  6  7  8  9 10 11 12 13 14 15 17 18 19]
duration [ 2  5 10 18]
price [ 2  5  7  9 10 11 18 19]
route [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 17 18 19]
row [ 5 10]
seat [ 2  5  7  9 10 18]
train [ 2  5  6  7  8  9 10 11 18 19]
type [10]
wagon [ 0  2  3  4  5  6  7  8  9 10 11 12 14 15 17 18 19]
zone [ 2  5  7  9 10 11 18]


In [108]:
order=np.argsort(lengths)
lengthssorted=np.sort(lengths)
lengthssorted

array([ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12., 13.,
       14., 15., 16., 17., 18., 19., 20.])

In [106]:
keyssorted=[keys[order[idx]] for idx in range(len(keys))]
keyssorted

['type',
 'row',
 'arrival platform',
 'duration',
 'arrival track',
 'seat',
 'zone',
 'price',
 'arrival location',
 'train',
 'departure location',
 'departure track',
 'departure platform',
 'departure date',
 'departure time',
 'departure station',
 'wagon',
 'class',
 'route',
 'arrival station']

In [125]:
candidatessorted=[candidates[order[idx]] for idx in range(len(candidates))]
for idx in range(len(candidatessorted)):
    print( idx,candidatessorted[idx])

0 [10]
1 [ 5 10]
2 [ 2  5 10]
3 [ 2  5 10 18]
4 [ 2  5  9 10 18]
5 [ 2  5  7  9 10 18]
6 [ 2  5  7  9 10 11 18]
7 [ 2  5  7  9 10 11 18 19]
8 [ 2  5  6  7  9 10 11 18 19]
9 [ 2  5  6  7  8  9 10 11 18 19]
10 [ 2  5  6  7  8  9 10 11 14 18 19]
11 [ 2  3  5  6  7  8  9 10 11 14 18 19]
12 [ 2  3  5  6  7  8  9 10 11 14 15 18 19]
13 [ 2  3  5  6  7  8  9 10 11 14 15 17 18 19]
14 [ 2  3  4  5  6  7  8  9 10 11 14 15 17 18 19]
15 [ 2  3  4  5  6  7  8  9 10 11 12 14 15 17 18 19]
16 [ 0  2  3  4  5  6  7  8  9 10 11 12 14 15 17 18 19]
17 [ 0  2  3  4  5  6  7  8  9 10 11 12 13 14 15 17 18 19]
18 [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 17 18 19]
19 [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]


In [134]:
# idxs on ticket according to keyssorted
ticketidxs=[]

usedidxs=[]

for idx in range(len(lengths)):
    cand=candidatessorted[idx]
    if idx>0:
        cand=cand[np.isin(cand,usedidxs)==False] # exclude used indices
    ticketidxs.append(cand[0])
    usedidxs.append(cand[0])

ticketidxs

[10, 5, 2, 18, 9, 7, 11, 19, 6, 8, 14, 3, 15, 17, 4, 12, 0, 13, 1, 16]

In [135]:
for k,t in zip(keyssorted,ticketidxs):
    print(k,t)

type 10
row 5
arrival platform 2
duration 18
arrival track 9
seat 7
zone 11
price 19
arrival location 6
train 8
departure location 14
departure track 3
departure platform 15
departure date 17
departure time 4
departure station 12
wagon 0
class 13
route 1
arrival station 16


In [143]:
# keys & corresponding ticket fields that start with departure
specialidxs=[idx for idx in range(len(keyssorted)) if "departure" in keyssorted[idx]]
specialkeys=[]
specialtickets=[]
for idx in specialidxs:
    specialkeys.append(keyssorted[idx])
    specialtickets.append(ticketidxs[idx])
    

In [151]:
specialtickets

[14, 3, 15, 17, 4, 12]

In [152]:
specialkeys

['departure location',
 'departure track',
 'departure platform',
 'departure date',
 'departure time',
 'departure station']

In [147]:
mysubset=np.array(mynumbers)[specialtickets]

In [149]:
specialkeys

['departure location',
 'departure track',
 'departure platform',
 'departure date',
 'departure time',
 'departure station']

In [153]:
mysubset

array([199, 179, 223, 181, 157, 227])

In [159]:
np.prod(mysubset.astype(np.float))

51240700105297.0

In [154]:
199*179*223*181*157*227

51240700105297